## Installing some of the dependencies 

In [3]:
! pip install psycopg2 
! pip install requests
! pip install SQLAlchemy

  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-macosx_10_15_x86_64.whl size=135420 sha256=5598901e1b955c3b360517648c65aa5c97c0497d1941927807aefe604a03ed50
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2


# Importing dependencies 

In [10]:
import pandas as pd
import numpy
import requests
import json
import csv

# Extraction Layer: Extracting the dataset from RapidAPI

In [17]:

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"2000"}

headers = {
	"x-rapidapi-key": "ce3693686amshd8a81ba62ce435ep1113afjsn137d2b9aa172",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()

# saving the data as a file

filename = 'propertyRecords.json'

# to write the file into the filename
with open (filename, 'w') as file:
    json.dump(data,file, indent = 4)

In [11]:
# to read into a dataframe

propertyrecords_df = pd.read_json('propertyRecords.json')


In [33]:
propertyrecords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   bedrooms          500 non-null    float64
 6   squareFootage     500 non-null    float64
 7   yearBuilt         500 non-null    float64
 8   features          500 non-null    object 
 9   county            500 non-null    object 
 10  assessorID        500 non-null    object 
 11  legalDescription  500 non-null    object 
 12  subdivision       500 non-null    object 
 13  zoning            500 non-null    object 
 14  bathrooms         500 non-null    float64
 15  lotSize           500 non-null    float64
 16  propertyType      500 non-null    object 
 1

In [27]:
# approach of cleaning
propertyrecords_df['features'] = propertyrecords_df['features'].apply(json.dumps)

# second approach to find and replace the nulls
propertyrecords_df.fillna({
        'bedrooms': 0,
        'addressLine2': 'Not available', 
        'squareFootage': 0,
        'yearBuilt': 0,
        'features': 'None',
        'assessorID': 'Unknown',
        'legalDescription': 'Not available',
        'subdivision': 'Not available', 
         'zoning': 'Unknown', 
         'bathrooms': 0, 
         'lotSize': 0,
         'propertyType': 'Unknown', 
         'taxAssessment': 'Not available',
        'propertyTaxes':  'Not available', 
         'lastSalePrice': 0,
        'lastSaleDate': 'Not available',
        'owner': 'Unknown',
        'ownerOccupied': 0,
        'addressLine2': 'Unknown',
        'county': 'Not available'}, inplace =True)

# to extract the id from the address
propertyrecords_df['id'] = propertyrecords_df['id'].apply(lambda x: id(x))
    
    

    



In [41]:
fact_columns = ['id','bedrooms', 'squareFootage','bathrooms', 'lotSize','lastSalePrice','lastSaleDate', 'longitude', 'latitude']
fact_table = propertyrecords_df[fact_columns]

In [29]:
fact_table

,id,bedrooms,squareFootage,bathrooms,lotSize,lastSalePrice,lastSaleDate
0,5008314160,2.0,1254.0,1.0,9800.0,9200.0,2017-03-01T00:00:00.000Z
1,5008308096,3.0,1438.0,2.0,6675.0,0.0,Not available
2,5008313200,3.0,1387.0,1.0,6098.0,330000.0,2018-09-28T00:00:00.000Z
3,5008308496,5.0,2210.0,2.0,9583.0,2264900.0,2023-09-11T00:00:00.000Z
4,5008308896,0.0,3720.0,0.0,15272.0,0.0,Not available
...,...,...,...,...,...,...,...
495,5010957424,3.0,1577.0,3.0,7623.0,229000.0,2020-04-28T00:00:00.000Z
496,5010957824,4.0,2266.0,2.5,8500.0,0.0,1998-08-05T00:00:00.000Z
497,5010958304,3.0,1520.0,2.0,8000.0,455000.0,2021-06-18T00:00:00.000Z
498,5010502640,3.0,1088.0,2.0,0.0,550000.0,2017-11-03T00:00:00.000Z


In [42]:
location_dim = propertyrecords_df[['addressLine2','county','zipCode','formattedAddress','state','city']].drop_duplicates().reset_index(drop=True)
location_dim.index.name = 'location_id'

In [43]:
location_dim

,addressLine2,county,zipCode,formattedAddress,state,city
location_id,,,,,,
0,Unknown,Lee,33905,"4413 Birmingham St, Fort Myers, FL 33905",FL,Fort Myers
1,Unknown,Harvey,67114,"900 Allison St, Newton, KS 67114",KS,Newton
2,Unknown,Riverside,92530,"258 White Oak Rd, Lake Elsinore, CA 92530",CA,Lake Elsinore
3,Unknown,Osceola,34741,"1202 Ponca Rd, Kissimmee, FL 34741",FL,Kissimmee
4,Unknown,Queens,11423,"86110 Marengo St, Hollis, NY 11423",NY,Hollis
...,...,...,...,...,...,...
495,Unknown,Newport News City,23608,"754 Flagship Dr, Newport News, VA 23608",VA,Newport News
496,Unknown,Dallas,75243,"9412 Meadowknoll Dr, Dallas, TX 75243",TX,Dallas
497,Unknown,Norfolk,2343,"28 Louise Rd, Holbrook, MA 02343",MA,Holbrook


In [ ]:
# to create the sales fact table
sale

In [49]:
feature_dim = propertyrecords_df[['id','features', 'propertyType', 'zoning']].drop_duplicates().reset_index(drop=True)
feature_dim.index.name = 'feature_id'

In [47]:
feature_dim

,features,propertyType,zoning
feature_id,,,
0,"""\""\\\""{\\\\\\\""cooling\\\\\\\"": true, \\\\\\\...",Land,RS-1
1,"""\""\\\""{\\\\\\\""architectureType\\\\\\\"": \\\\...",Single Family,R-1
2,"""\""\\\""{\\\\\\\""cooling\\\\\\\"": true, \\\\\\\...",Single Family,R3
3,"""\""\\\""{}\\\""\""""",Land,Unknown
4,"""\""\\\""{\\\\\\\""architectureType\\\\\\\"": \\\\...",Duplex-Triplex,R1-2
...,...,...,...
353,"""\""\\\""{\\\\\\\""architectureType\\\\\\\"": \\\\...",Single Family,R4
354,"""\""\\\""{\\\\\\\""cooling\\\\\\\"": true, \\\\\\\...",Single Family,Z298
355,"""\""\\\""{\\\\\\\""architectureType\\\\\\\"": \\\\...",Single Family,R3
